![alt text for screen readers](https://intro-to-btt-using-python-assets.s3.amazonaws.com/bladesight_logo_horizontal_ORIGINAL.jpg).
## Chapter 2: Data zeroing and Filtering 

## Dependencies

In [ ]:
# Run this cell if you have not installed the `bladesight` package yet
%pip install bladesight
## NBNBNB! You may need to restart the kernel after installing the package! If you 
# installed it through the Kernel, you can skip this cell.

In [ ]:
# If plotly is not installed
%pip install plotly
## NBNBNB! You may need to restart the kernel after installing the package! If you 
# installed it through the Kernel, you can skip this cell.

In [ ]:
# If Numba is not installed
%pip install numba
## NBNBNB! You may need to restart the kernel after installing the package! If you 
# installed it through the Kernel, you can skip this cell.

## Imports

In [ ]:
# Import the Datasets handler from the bladesight package
from bladesight import Datasets
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
# Load the dataset
ds_ch2 = Datasets["data/intro_to_btt/intro_to_btt_ch02"]

df_proximity_probe = ds_ch2["table/three_generated_pulses"]

## Vectorized Implementation

In [ ]:
TRIGGER_ON_RISING_EDGE = True
THRESHOLD_LEVEL = 0.4 # Volts

if TRIGGER_ON_RISING_EDGE:
    sr_threshold_over = (df_proximity_probe['data'] >= THRESHOLD_LEVEL).astype(int)
else:
    sr_threshold_over = (df_proximity_probe['data'] <= THRESHOLD_LEVEL).astype(int)

diff_sr_threshold = sr_threshold_over.diff()

diff_sr_threshold = diff_sr_threshold.bfill()

sr_threshold_change = diff_sr_threshold > 0

sr_toas = df_proximity_probe['time'][sr_threshold_change]

In [ ]:
print("THE TOAs ARE:")
print(sr_toas.values)

## Visualize the three pulses and the algorithm

In [ ]:
df_proximity_probe['over?'] = sr_threshold_over
df_proximity_probe['diff'] = diff_sr_threshold

In [ ]:
# Plot the waveform with the over indicator
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
        go.Scatter(x=df_proximity_probe['time'], y=df_proximity_probe['data'], name='Voltage signal'),
        secondary_y=False,
)
fig.add_trace(
        go.Scatter(x=df_proximity_probe['time'], y=df_proximity_probe['over?'], name='Over threshold?'),
        secondary_y=True,
)
fig.add_trace(
        go.Scatter(x=df_proximity_probe['time'], y=df_proximity_probe['diff'], name='Diff'),
        secondary_y=True,
)
fig.update_layout(
    title="Voltage signal with threshold over indicator",
    xaxis_title="Time (s)",
    yaxis_title="Voltage (V)",
)
fig.show()



## Sequential implementation

In [ ]:
# Import the libraries for the sequential search
from numba import njit
from bladesight import Datasets
import numpy as np
from typing import Optional

In [ ]:
@njit
def seq_simple_threshold_crossing(
    arr_t : np.ndarray,
    arr_s : np.ndarray,
    threshold : float,
    n_est : Optional[float] = None,
    trigger_on_rising_edge : bool = True
) -> np.ndarray:
    """ A simple sequential threshold crossing algorithm.

    Args:
        arr_t (np.ndarray): The array containing the time values.
        arr_s (np.ndarray): The array containing the signal voltage values 
            corresponding to the time values.
        threshold (float): The threshold value.
        n_est (float, optional): The estimated number of ToAs in this signal. Defaults to None.
            This number is used to pre-allocate the array containing the ToAs. If this number is
            not provided, the array will be pre-allocated as the same dimension as arr_t and arr_s.
        trigger_on_rising_edge (bool, optional): Whether to trigger ToAs on the rising or falling 
            edge. Defaults to True. If True, the ToA is triggered on the rising edge.
    
    Returns:
        np.ndarray: An array containing the ToAs.
    """
    
    # Pre-allocate the array containing the ToAs
    if n_est is None:
        arr_toa = -1 * np.ones(arr_t.shape)
    else:
        arr_toa = -1 * np.ones(n_est)

    # Initialise the index of the ToA array
    i_toa = 0

    # Initialise the previous sample value
    prev_sample = arr_s[0]

    # Loop through all the samples
    for i_sample in range(1, arr_s.shape[0]):

        # Get the current sample value
        curr_sample = arr_s[i_sample]

        # Check if the threshold is crossed
        if trigger_on_rising_edge:
            if (prev_sample < threshold) and (curr_sample >= threshold):
                arr_toa[i_toa] = arr_t[i_sample]
                i_toa += 1
        else:
            if (prev_sample > threshold) and (curr_sample <= threshold):
                arr_toa[i_toa] = arr_t[i_sample]
                i_toa += 1

        # Update the previous sample value
        prev_sample = curr_sample

    # Return the array containing the ToAs
    return arr_toa[:i_toa]


In [ ]:
toas = seq_simple_threshold_crossing(df_proximity_probe['time'].values, df_proximity_probe['data'].values, 0.4)
print("THE TOAs ARE:")
print(toas)

### Interpolate voltage

In [ ]:
@njit
def seq_threshold_crossing_interp(
    arr_t : np.ndarray,
    arr_s : np.ndarray,
    threshold : float,
    n_est : Optional[float] = None,
    trigger_on_rising_edge : bool = True
) -> np.ndarray:
    """ A sequential threshold crossing algorithm that interpolates
        the ToA between the two samples where the signal crosses 
        the threshold.

    Args:
        arr_t (np.ndarray): The array containing the time values.
        arr_s (np.ndarray): The array containing the signal voltage values 
            corresponding to the time values.
        threshold (float): The threshold value.
        n_est (float, optional): The estimated number of ToAs in this signal. Defaults to None.
            This number is used to pre-allocate the array containing the ToAs. If this number is
            not provided, the array will be pre-allocated as the same dimension as arr_t and arr_s.
        trigger_on_rising_edge (bool, optional): Whether to trigger ToAs on the rising or falling 
            edge. Defaults to True. If True, the ToA is triggered on the rising edge.
    
    Returns:
        np.ndarray: An array containing the ToAs.
    """
        
    # Pre-allocate the array containing the ToAs
    if n_est is None:
        arr_toa = -1 * np.ones(arr_t.shape)
    else:
        arr_toa = -1 * np.ones(n_est)

    # Initialise the index of the ToA array
    i_toa = 0

    # Initialise the previous sample value
    prev_sample = arr_s[0]

    # Loop through all the samples
    for i_sample in range(1, arr_s.shape[0]):

        # Get the current sample value
        curr_sample = arr_s[i_sample]

        # Check if the threshold is crossed
        if trigger_on_rising_edge:
            if (prev_sample < threshold) and (curr_sample >= threshold):
                # Interpolate the ToA
                arr_toa[i_toa] = (
                    arr_t[i_sample - 1] 
                    + (arr_t[i_sample] - arr_t[i_sample - 1]) 
                    * (threshold - prev_sample) 
                    / (curr_sample - prev_sample)
                )
                i_toa += 1
        else:
            if (prev_sample > threshold) and (curr_sample <= threshold):
                # Interpolate the ToA
                arr_toa[i_toa] = (
                    arr_t[i_sample - 1] 
                    + (arr_t[i_sample] - arr_t[i_sample - 1]) 
                    * (threshold - prev_sample) 
                    / (curr_sample - prev_sample)
                )
                i_toa += 1

        # Update the previous sample value
        prev_sample = curr_sample

    # Return the array containing the ToAs
    return arr_toa[:i_toa]

In [ ]:
toas = seq_threshold_crossing_interp(df_proximity_probe['time'].values, df_proximity_probe['data'].values, 0.4)
print("THE TOAs ARE:")
print(toas)

### Too much noise present

In [ ]:
df_proximity_probe_noisy = ds_ch2["table/three_generated_pulses_noisy"]
toas_simple = seq_threshold_crossing_interp(df_proximity_probe_noisy['time'].values, df_proximity_probe_noisy['data'].values, 0.4)
print("THE TOAs ARE:")
print(toas_simple)

### Include hysteresis

In [ ]:
@njit
def seq_threshold_crossing_hysteresis_pos(
    arr_t : np.ndarray,
    arr_s : np.ndarray,
    threshold : float,
    hysteresis_height : float,
    n_est : Optional[float] = None,
) -> np.ndarray:
    """ A sequential threshold crossing algorithm that interpolates
        the ToA between the two samples where the signal crosses 
        the threshold.

    Args:
        arr_t (np.ndarray): The array containing the time values.
        arr_s (np.ndarray): The array containing the signal voltage values 
            corresponding to the time values.
        threshold (float): The threshold value.
        hysteresis_height (float): The height of the hysteresis, in the same
            units as the signal.
        n_est (float, optional): The estimated number of ToAs in this signal. Defaults to None.
            This number is used to pre-allocate the array containing the ToAs. If this number is
            not provided, the array will be pre-allocated as the same dimension as arr_t and arr_s.
    
    Returns:
        np.ndarray: An array containing the ToAs.
    """
    threshold_lower = threshold - hysteresis_height
    trigger_state = True if arr_s[0] > threshold_lower else False

    # Pre-allocate the array containing the ToAs
    if n_est is None:
        arr_toa = -1 * np.ones(arr_t.shape)
    else:
        arr_toa = -1 * np.ones(n_est)

    # Initialise the index of the ToA array
    i_toa = 0

    # Initialise the previous sample value
    prev_sample = arr_s[0]

    # Loop through all the samples
    for i_sample in range(1, arr_s.shape[0]):
        # Get the current sample value
        curr_sample = arr_s[i_sample]

        # Check if the threshold is crossed
        if trigger_state is True:
            if curr_sample <= threshold_lower:
                trigger_state = False
        else:
            if curr_sample >= threshold:
                trigger_state = True
                # Interpolate the ToA
                arr_toa[i_toa] = (
                    arr_t[i_sample - 1] 
                    + (arr_t[i_sample] - arr_t[i_sample - 1]) 
                    * (threshold - prev_sample) 
                    / (curr_sample - prev_sample)
                )
                i_toa += 1

        # Update the previous sample value
        prev_sample = curr_sample

    # Return the array containing the ToAs
    return arr_toa[:i_toa]


In [ ]:
toas = seq_threshold_crossing_hysteresis_pos(df_proximity_probe_noisy['time'].values, df_proximity_probe_noisy['data'].values, 0.4, 0.2)
print("THE TOAs ARE:")
print(toas)

In [ ]:
# Load a real signal
df_alum_blisk = ds_ch2['table/aluminium_blisk_1200_rpm']
df_alum_blisk

In [ ]:
# Create a time array. The signal was acquired at a rate of 2e6
t_arr = np.arange(df_alum_blisk.shape[0]) * 1/(2e6)

In [ ]:
taos = seq_threshold_crossing_hysteresis_pos(
    t_arr,
    df_alum_blisk['volt'].values,
    -5.5, # Threshold value. The signals are negative
    0.2,# Hysteresis width
    n_est=int(5*1200/60*10*2)
)

In [ ]:
tut_02_dataset = Datasets["data/intro_to_btt/intro_to_btt_ch02"]

In [ ]:
df_proximity_probe = tut_02_dataset['table/aluminium_blisk_1200_rpm']

### Vizualize the first 5 extracted ToAs

In [ ]:
fig = go.Figure()
i_start = 15000
i_end = 120000
fig.add_trace(go.Scattergl(x=t_arr[i_start:i_end], y=df_alum_blisk['volt'].values[i_start:i_end]))
fig.add_trace(go.Scattergl(x=taos[:5], y=np.ones(5)*-5.5, mode='markers'))
fig.show()

### Performance of sequential implementation

In [ ]:
%%timeit
taos = seq_threshold_crossing_hysteresis_pos(
    t_arr,
    df_alum_blisk['volt'].values,
    -5.5,
    0.2,
    n_est=int(5*1200/60*10*2)
)

### Performance of vectorized implementation

In [ ]:
df_alum_blisk["time"] = t_arr

In [ ]:
%%timeit
TRIGGER_ON_RISING_EDGE = True
THRESHOLD_LEVEL = -5.5 # Volts

if TRIGGER_ON_RISING_EDGE:
    sr_threshold_over = (df_alum_blisk['volt'] >= THRESHOLD_LEVEL).astype(int)
else:
    sr_threshold_over = (df_alum_blisk['volt'] <= THRESHOLD_LEVEL).astype(int)

diff_sr_threshold = sr_threshold_over.diff()

diff_sr_threshold = diff_sr_threshold.bfill()

sr_threshold_change = diff_sr_threshold > 0

sr_toas = df_alum_blisk['time'][sr_threshold_change]

In [ ]:
print(sr_toas)